## Twinbot  with Tools

In this notebook, we demonstrate how to enhance the Twinbot's capabilities by integrating tool use. Specifically, we implement two tools: one for recording unknown questions and another for capturing user contact details. These tools enable the Twinbot to handle situations where it lacks information and to facilitate user engagement effectively.

We will use the Pushover service to send notifications for both unknown questions and user contact details. This allows us to receive real-time alerts on our devices whenever the Twinbot encounters an unknown question or when a user expresses interest in getting in touch.

### Setup

In [2]:
# imports

from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

In [ ]:
load_dotenv(override=True)

True

In [50]:
OLLAMA_API_KEY = os.getenv('OLLAMA_API_KEY')
OLLAMA_BASE_URL = os.getenv('OLLAMA_BASE_URL')

MODEL_PHI = os.getenv('MODEL_PHI4_14B')
MODEL_LLAMA = os.getenv('MODEL_LLAMA3_8B')

try:
    print(f"- OLLAMA_API_KEY = {OLLAMA_API_KEY}")
    print(f"- OLLAMA_BASE_URL = {OLLAMA_BASE_URL}")
    print(f"- MODEL_PHI = {MODEL_PHI}")
    print(f"- MODEL_LLAMA = {MODEL_LLAMA}")

except Exception as e:
    print(f"ERROR: One or more parameteres were not set.")

- OLLAMA_API_KEY = ollama
- OLLAMA_BASE_URL = http://localhost:11434/v1
- MODEL_PHI = phi4
- MODEL_LLAMA = llama3.1


In [6]:
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key=OLLAMA_API_KEY)
phi = OpenAI(base_url=OLLAMA_BASE_URL, api_key=OLLAMA_API_KEY)

#### Pushover

Pushover is a nifty tool for sending Push Notifications to your phone.

It's super easy to set up and install!

Simply visit https://pushover.net/ and click 'Login or Signup' on the top right to sign up for a free account, and create your API keys.

Once you've signed up, on the home screen, click "Create an Application/API Token", and give it any name (like Agents) and click Create Application.

Then add 2 lines to your `.env` file:

PUSHOVER_USER=_put the key that's on the top right of your Pushover home screen and probably starts with a u_  
PUSHOVER_TOKEN=_put the key when you click into your new application called Agents (or whatever) and probably starts with an a_

Remember to save your `.env` file, and run `load_dotenv(override=True)` after saving, to set your environment variables.

Finally, click "Add Phone, Tablet or Desktop" to install on your phone.



In [ ]:
# For pushover
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

if pushover_user:
    print(f"Pushover user found and starts with {pushover_user[0]}")
else:
    print("Pushover user not found")

if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("Pushover token not found")

Pushover user found and starts with u
Pushover token found and starts with a


In [31]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [32]:
push("HEY!!")

Push: HEY!!


### Tool functions for LLMs

In [33]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

In [34]:
def record_unknown_question(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

#### JSONs for Tools

Blobs of JSONs that define tools (defined above) for LLMs to use.

In [35]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [36]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

#### Tools List

In [37]:
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [38]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Use this tool to record that a user is interested in being in touch and provided an email address',
   'parameters': {'type': 'object',
    'properties': {'email': {'type': 'string',
      'description': 'The email address of this user'},
     'name': {'type': 'string',
      'description': "The user's name, if they provided it"},
     'notes': {'type': 'string',
      'description': "Any additional information about the conversation that's worth recording to give context"}},
    'required': ['email'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': "The question that couldn't be answered"}},
    'required': ['quest

#### Handle tool calls

We define a helper function to handle tool calls from the LLM.

```python
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        # THE BIG IF STATEMENT!!!
        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)

        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results
```

However, since keeping track of many tool names and their corresponding functions can get tedious, we use a more scalable approach using `globals()` to dynamically map tool names to functions.

In [40]:
globals()["record_unknown_question"]("this is a really hard question")

Push: Recording this is a really hard question asked that I couldn't answer


{'recorded': 'ok'}

In [ ]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results


## Implementing Tools 

In [ ]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Nellie"

In [43]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:

        # Call LLM passing in the tools json
        response = ollama.chat.completions.create(model=MODEL_LLAMA, messages=messages, tools=tools)
        finish_reason = response.choices[0].finish_reason
        
        # Filter the response to use tool calls if needed
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

## Launching the Twinbot with Tools

Now we launch our Tool-Using Twinbot!

```python
gr.ChatInterface(chat, type="messages").launch()
```

For the initial interaction, the Twinbot will respond normally. There is no tool use.

![Tool Call - Normal flow](../img/twinbot-hi.png)

However, when we ask a question that the Twinbot does not know the answer to, it will call the `record_unknown_question` tool to log the question for later review.

![Tool Call - Pushover for unknown question](../img/pushover-unk-question.png)

When the Twinbot detects an intention to get in touch and an email is provided, the Twinbot will call the `record_user_details` tool to log our contact information via Pushover.

![Tool Call - Pushover for contact details](../img/pushover-email.png)


Below we see the logs for both tool calls:
```text
Logs:
Tool called: record_unknown_question
Push: Recording Who's your favorite artist? asked that I couldn't answer

Tool called: record_user_details
Push: Recording interest from  with email example@email.com and notes User wants to get in touch
```

And here are the notifications we received on our phone via Pushover:

![Pushover Notifications](../img/pushover-twinbot-notifications.png)

## Deployment of our Twinbot with Tools

We will deploy to **HuggingFace Spaces**. We need an 'app.py' file (which we have) to deploy.

Before you start: remember to update the files in the "me" directory - your LinkedIn profile and summary.txt - so that it talks about you! Also change the name in `self.name` in `app.py`.

Also check that there's no README file within the current directory. If there is one, please delete it. The deploy process creates a new README file in this directory for you.



### Initial setup

Follow the steps if you haven't done this before:
1. Visit https://huggingface.co and set up an account  
2. From the Avatar menu on the top right, choose Access Tokens. Choose "Create New Token". Give it WRITE permissions - it needs to have WRITE permissions! Keep a record of your new key.  
3. In the Terminal, run: `uv tool install 'huggingface_hub[cli]'` to install the HuggingFace tool, then `hf auth login` to login at the command line with your key. Afterwards, run `hf auth whoami` to check you're logged in  
4. Take your new token and add it to your .env file: `HF_TOKEN=hf_xxx` for the future
5. From the current directory, enter: `uv run gradio deploy` 
6. Follow its instructions: 
    - name it "career_conversation", or whatever you like that's available,
    - specify app.py, 
    - choose cpu-basic as the hardware, 
    - say Yes to needing to supply secrets, 
    - provide your openai api key, 
    - your pushover user and token, 
    - and say "no" to github actions.  

Credits: Robert, James, Martins, Andras and Priya for these tips.  
Please read the next 2 sections - how to change your Secrets, and how to redeploy your Space (you may need to delete the README.md that gets created in this directory).

#### More about these secrets:

If you're confused by what's going on with these secrets: it just wants you to enter the key name and value for each of your secrets -- so you would enter:  
`OPENAI_API_KEY`  
Followed by:  
`sk-proj-...`  

And if you don't want to set secrets this way, or something goes wrong with it, it's no problem - you can change your secrets later:  
1. Log in to HuggingFace website  
2. Go to your profile screen via the Avatar menu on the top right  
3. Select the Space you deployed  
4. Click on the Settings wheel on the top right  
5. You can scroll down to change your secrets (Variables and Secrets section), delete the space, etc.

*If you want to completely replace everything and start again with your keys, you may need to delete the README.md that got created in this folder.*

### And now you should be deployed!

- Here is the original from the course: https://huggingface.co/spaces/ed-donner/Career_Conversation

For more information on deployment:
https://www.gradio.app/guides/sharing-your-app#hosting-on-hf-spaces

To delete your Space in the future:  
1. Log in to HuggingFace
2. From the Avatar menu, select your profile
3. Click on the Space itself and select the settings wheel on the top right
4. Scroll to the Delete section at the bottom
5. ALSO: delete the README file that Gradio may have created inside this 1_foundations folder (otherwise it won't ask you the questions the next time you do a gradio deploy)


## Twinbot Demo

**[Professional Twin Chatbot](https://huggingface.co/spaces/cordovank/career_twinbot)** is deployed on Hugging Face Spaces!

![Career Twinbot Demo App](../img/hf-career-twinbot.png)